# Tissue nuclear analysis for wangyu

### Cellpose model segment every nuclei. Quantify intensities.

### 03/04/24
### Author: Andy D. Tran, CCR Microscopy Core, LCBG, CCR, NCI

In [1]:
# Import libraries-----------------------------------------------------------------------------------------------

import os 
import numpy as np 
import pandas as pd 
#import napari 
import re 

from scipy import ndimage as nd 
from aicsimageio import AICSImage
from tifffile import imread, imwrite
from skimage.measure import regionprops
from tqdm import tqdm
from cellpose import models

In [2]:
# Define functions------------------------------------------------------------------------------------------------

def model_apply(model, img):
    mask, flows, style = model.eval(img, channels = [0, 0])
    
    return mask

def roi_quant(cell_img, green_img, cd31_img):
    df = pd.DataFrame() 
    
    cell_id = []
    area = [] 
    green_int = [] 
    cd31_int = [] 
    
    green_props = regionprops(cell_img, intensity_image = green_img)
    cd31_props = regionprops(cell_img, intensity_image = cd31_img)
    
    for roi in tqdm(range(len(green_props))):
        cell_id.append(green_props[roi].label)
        area.append(green_props[roi].area)
        green_int.append(green_props[roi].mean_intensity)
        cd31_int.append(cd31_props[roi].mean_intensity)
        
    df['cell_id'] = cell_id
    df['area'] = area
    df['green_int'] = green_int
    df['cd31_int'] = cd31_int
    
    return df 

class iterator:
  def __iter__(self):
    self.a = 1
    return self

  def __next__(self):
    x = self.a
    self.a += 1
    return x

In [3]:
cell_model = models.CellposeModel(gpu = True, pretrained_model = '/data/tranad/image/wangyu/training/nuc/models/wangyu_nuc_02')

In [176]:
base_path = '/data/tranad/image/wangyu/images/12_28'

In [177]:
tmp_list = os.listdir(base_path)

img_list = [] 
for tmp in tmp_list:
    if re.search('.czi', tmp):
        img_list.append(tmp)
        
print(img_list)
print(len(img_list))

['B16F10-shSHP2-C5-Ki67-CD31.czi', 'B16F10-shSHP2-GDC4-Ki67-CD31.czi', 'B16F10-shSHP2-GDC3-FITC-dextran-Ki67-CD31.czi', 'B16F10-shSHP2-C4-FITC-dextran-Ki67-CD31.czi', 'MS1.czi', 'B16F10-shSHP2-GDC5-Ki67-CD31.czi', 'MS2.czi']
7


In [ ]:
iterclass = iterator()
index = iter(iterclass)

for img_name in img_list:
    print(img_name)
    print(next(index), '/', len(img_list))
    
    img_path = os.path.join(base_path, img_name)
    img_tmp = AICSImage(img_path)
    print(img_tmp.shape)
    print(img_tmp.channel_names)
    
    img = img_tmp.data
    
    cd31_img = img[0, 0, 0, :, :]
    green_img = img[0, 1, 0, :, :]
    dapi_img = img[0, 2, 0, :, :]
    
    cell_img = model_apply(cell_model, dapi_img)
    cell_img = np.uint16(cell_img)
    cell_path = os.path.join(base_path, img_name.replace('.czi', '_cell.tif'))
    imwrite(cell_path, cell_img)
    
    cell_df = roi_quant(cell_img, green_img, cd31_img)
    df_path = os.path.join(base_path, img_name.replace('.czi', '_df.csv'))
    cell_df.to_csv(df_path, header = True)

### Test

In [199]:
#img_name = img_list[4]
img_name = 'B16F10-shSHP2-GDC5-Ki67-CD31.czi'
print(img_name)

img_path = os.path.join(base_path, img_name)
img_tmp = AICSImage(img_path)
print(img_tmp.shape)
print(img_tmp.channel_names)

B16F10-shSHP2-GDC5-Ki67-CD31.czi
(1, 3, 1, 3557, 2251)
['ChS2-T2', 'ChS1-T3', 'Ch1-T4']


In [200]:
img = img_tmp.data

cd31_img = img[0, 1, 0, :, :]
green_img = img[0, 2, 0, :, :]
dapi_img = img[0, 2, 0, :, :]

print(dapi_img.shape)

(3557, 2251)


In [201]:
cell_img = model_apply(cell_model, dapi_img)

In [202]:
output_path = '/data/tranad/image/wangyu/output/12_28'

cell_img = np.uint16(cell_img)
cell_path = os.path.join(output_path, img_name.replace('.czi', '_cell.tif'))
imwrite(cell_path, cell_img)

In [203]:
cell_df = roi_quant(cell_img, green_img, cd31_img)

100%|████████████████████████████████████| 2626/2626 [00:00<00:00, 14944.58it/s]


In [204]:
df_path = os.path.join(output_path, img_name.replace('.czi', '_df.csv'))
cell_df.to_csv(df_path, header = True)

In [205]:
print(cell_img.shape)

(3557, 2251)
